Código Otimizado para utilizar com a GPU do Google Colab

In [1]:
# CÉLULA 1 - MONTAGEM DO GOOGLE DRIVE E DEFINIÇÃO DE CAMINHOS GLOBAIS
from google.colab import drive #importa biblioteca para conectar Colab com Drive (onde o database está)
drive.mount('/content/drive') #acessa meu Drive como uma pasta local

# Defina os caminhos para suas pastas e arquivos no Google Drive
  #Armazena na variável o caminho do database
DATA_PATH = "/content/drive/MyDrive/RAG/databaseRag/all"
  #Armazena na variável o caminho do model LLM (que está no drive)
LLAMA_MODEL_PATH = "/content/drive/MyDrive/RAG/models/new/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf"
  #Armazena na variável o caminho onde o banco ChromaDB será salvo. O Chroma vai guardar os vetores numéricos que eram os arigos
CHROMA_PERSIST_DIR = "/content/drive/MyDrive/RAG/databaseRag/chroma_db_new"

#Printar na tela os valores das variáveis (útil para verificação, se foi linkado com sucesso)
print(f"Caminho dos Dados: {DATA_PATH}")
print(f"Caminho do Modelo LLM: {LLAMA_MODEL_PATH}")
print(f"Caminho do Banco de Dados Chroma: {CHROMA_PERSIST_DIR}")

Mounted at /content/drive
Caminho dos Dados: /content/drive/MyDrive/RAG/databaseRag/all
Caminho do Modelo LLM: /content/drive/MyDrive/RAG/models/new/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf
Caminho do Banco de Dados Chroma: /content/drive/MyDrive/RAG/databaseRag/chroma_db_new


In [2]:
#CÉLULA 2 - REMOÇÃO DO ANTIGO BD

#importa biblioteca para interagir com o sistema operacional
import os
#importa a biblioteca para fazer operações de alto nível em arquivos, como apagar um diretório inteiro
import shutil


#verifica se a pasta do banco de dados ChromaDB (definida na céula 1) já existe
if os.path.exists(CHROMA_PERSIST_DIR): #caso exista
    print(f"Removendo o banco de dados antigo em: {CHROMA_PERSIST_DIR}") #informa que será excluida
    shutil.rmtree(CHROMA_PERSIST_DIR) #apaga a pasta e o conteúdo que houver dentro dela
    print("Banco de dados antigo removido com sucesso.")
else: #caso não exista, apenas informa:
    print("Nenhum banco de dados antigo encontrado para remover.")

Removendo o banco de dados antigo em: /content/drive/MyDrive/RAG/databaseRag/chroma_db_new
Banco de dados antigo removido com sucesso.


In [3]:
# CÉLULA 3 - INSTALAÇÕES
print("⬇️ 1. Instalando bibliotecas base...")

# Instalando tanto PyPDF quanto PDFMiner para não ter divergência
!pip install -q pypdf docx2txt sentence-transformers
!pip install -q pdfminer.six

print("⬇️ 2. Instalando Banco de Dados e LangChain...")
# Seta a versão estável (mesma que foi utilizada na primeira versão do código)
!pip install -q chromadb==0.4.22
!pip install -q langchain==0.1.0 langchain-community==0.0.10 langchain-core==0.1.8 --no-deps

print("⬇️ 3. Instalando Llama-cpp (Aceleração por GPU T4)...")
!pip install llama-cpp-python --no-cache-dir --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121

print("\n✅ Ambiente pronto!")

⬇️ 1. Instalando bibliotecas base...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 56.3 MB/s eta 0:00:00
⬇️ 2. Instalando Banco de Dados e LangChain...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.2 MB/s 

In [4]:
# CÉLULA 4 - DEMAIS IMPORTAÇÕES

#biblioteca para interagir com arquivos e pastas do sistema
import os
import shutil
#importa divisor de texto, que quebra documentos longos em chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
#importa a classe do ChromaDB, onde os chunks serão armazenados e indexados
from langchain_community.vectorstores import Chroma
#importa um fluxo pré construido que conecta a busca de documentos com geração de respostas pelo LLM
from langchain.chains import RetrievalQA
#importa o conector que permite ao LangChain usar o modelo Llama 3 do arquivo .guff
from langchain_community.llms import LlamaCpp
#importa ferramenta que cria templates de prompt, estrtuturando a pergunta enviada ao LLM
from langchain.prompts import PromptTemplate
#importa função para limpar metadados complexos
from langchain_community.vectorstores.utils import filter_complex_metadata
#carregador de arquivos unstructured, possibilita a leitura de arquivos com diretos formatos **ponto chave p ler artigos cientificos e revistas
from langchain_community.document_loaders import UnstructuredFileLoader
#importa 'callback' para exibir a resposta do LLM em tempo real ~vai printando cfme encontra resposta~
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
#importa classe principal que carrega modelo para transformar trechos de texto em vetores numéricos (embeddings)
from langchain_community.embeddings import SentenceTransformerEmbeddings


In [5]:
# CÉLULA 5 - FUNÇÕES DE CARREGAMENTO (PDFMiner) E CHUNKING
import os
from langchain_community.document_loaders import PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_documents_from_folder(folder_path):
    documents = []

    # Listagem de arquivos
    files = [f for f in os.listdir(folder_path) if f.endswith('.pdf') and not f.startswith("~$")]

    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
            print(f"Carregando com PDFMiner: {file}...")
            loader = PDFMinerLoader(file_path)
            documents.extend(loader.load())
        except Exception as e:
            print(f"❌ Erro ao processar {file}: {e}")

    print(f"\n📄 Total de documentos carregados: {len(documents)}")
    return documents

def split_documents_into_chunks(documents, chunk_size=800, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"🧩 Gerados {len(chunks)} chunks.")
    return chunks

In [6]:
# CÉLULA 6 - CRIAÇÃO DO BANCO CHROMA
import numpy as np

# Patch para NumPy 2.x
if not hasattr(np, 'float_'): np.float_ = np.float64

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

def create_vector_db():
    # 1. Definição de caminhos
    DATA_PATH = "/content/drive/MyDrive/RAG/databaseRag/artigos"
    CHROMA_PERSIST_DIR = "/content/drive/MyDrive/RAG/chroma_db" # Garanta que esta variável esteja definida

    # 2. Reutilizando as funções da Célula 5
    print(f"🚀 Iniciando processamento de documentos em: {DATA_PATH}")
    all_documents = load_documents_from_folder(DATA_PATH)

    # 3. Chunking
    chunks = split_documents_into_chunks(all_documents, chunk_size=1000, chunk_overlap=250)

    # 4. Criação e Persistência do Banco
    print("🧠 Gerando Embeddings e salvando no Chroma...")
    embedding_function = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    vector_db = Chroma.from_documents(
        documents=chunks,
        embedding=embedding_function,
        persist_directory=CHROMA_PERSIST_DIR
    )

    vector_db.persist()

    print(f"✅ SUCESSO! Banco de dados criado e salvo em: {CHROMA_PERSIST_DIR}")
    return vector_db

# Execução
vector_db = create_vector_db()

🚀 Iniciando processamento de documentos em: /content/drive/MyDrive/RAG/databaseRag/artigos
Carregando com PDFMiner: 5371-Texto do artigo-16016-1-10-20081007.pdf...


Carregando com PDFMiner: 4284-28572-1-PB.pdf...
Carregando com PDFMiner: IIDD225 PDF.pdf...
Carregando com PDFMiner: rgenfermagem,+551-556.pdf...
Carregando com PDFMiner: 35284-Texto do Artigo-253541-1-10-20241114.pdf...
Carregando com PDFMiner: ijrb-17-883.pdf...
Carregando com PDFMiner: ali,+2+CUIDADOS+DE+ENFERMAGEM+A+PACIENTES+EM+PRÉ-OPERATÓRIO+PROPOSTA+DE+CHECKLIST.pdf...
Carregando com PDFMiner: afab146.pdf...
Carregando com PDFMiner: submetido-ARTIGO+CIRURGIA+CARDÍACA (1).pdf...
Carregando com PDFMiner: download.pdf...
Carregando com PDFMiner: scopusresults.pdf...
Carregando com PDFMiner: artigo scopus.pdf...
Carregando com PDFMiner: artigo download.pdf...

📄 Total de documentos carregados: 13
🧩 Gerados 575 chunks.
🧠 Gerando Embeddings e salvando no Chroma...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


✅ SUCESSO! Banco de dados criado e salvo em: /content/drive/MyDrive/RAG/chroma_db


In [ ]:
# CÉLULA FINAL - CHATBOT RAG (VERSÃO ALTA PERFORMANCE GPU)
import os
import glob
import gradio as gr
import numpy as np

# Patch para compatibilidade de versões
if not hasattr(np, 'float_'): np.float_ = np.float64

from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

print("🚀 Iniciando CardioBot com Aceleração por GPU T4...")

# 1. Localização do Modelo
base_path = "/content/drive/MyDrive/RAG"
model_files = glob.glob(os.path.join(base_path, "**/Meta-Llama-3-8B-Instruct-Q4_K_M.gguf"), recursive=True)

if not model_files:
    raise FileNotFoundError("❌ Modelo .gguf não encontrado. Verifique se o Drive está montado.")

model_path = model_files[0]

# 2. Configuração do Modelo (Foco em Velocidade e Rigor)
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=-1,      # Carrega todas as camadas na GPU
    n_batch=512,
    n_ctx=4096,           # Janela de contexto ampla
    temperature=0.1,      # Baixa temperatura para evitar invenções
    repeat_penalty=1.1,
    stop=["<|eot_id|>", "CONTEXTO:", "PERGUNTA:"],
    verbose=False
)

# 3. Conexão com o Banco de Dados (Use o caminho onde você salvou o banco)
# Se você criou o banco localmente na Célula 8, use: "/content/db_cardio_local"
CHROMA_PATH = "/content/db_cardio_local"
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

# 4. Template Unificado (Rigor Técnico e Sem Saudações)
template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Você é um profissional de saúde, especialista em orientação pré e pós-operatória de cirurgia cardíaca.
Forneça respostas padronizadas, seguras, objetivas e em linguagem acessível ao paciente. Suas orientações são estritamente baseadas no contexto fornecido.

Regra de Segurança: Se a informação for individualizada, ou não estiver no contexto, a resposta deve ser:
"Esta é uma questão específica do seu histórico médico e deve ser discutida diretamente com seu médico cardiologista ou a equipe de enfermagem."
OU "Essa informação depende de uma avaliação individualizada. Por favor, consulte seu médico ou a equipe de saúde responsável.

5. Responda sempre em Português do Brasil.<|eot_id|><|start_header_id|>user<|end_header_id|>
CONTEXTO:
{context}

PERGUNTA:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

# 5. Função de Resposta
def responder(pergunta, historico):
    if not pergunta: return ""
    resultado = qa_chain.invoke({"query": pergunta})
    resposta = resultado["result"].strip()

    # Limpeza final de tags do Llama 3
    if "assistant" in resposta:
        resposta = resposta.split("assistant")[-1].strip()

    return resposta.replace("<|eot_id|>", "").strip()

# 6. Interface
demo = gr.ChatInterface(
    fn=responder,
    title="🫀 CardioBot - Orientações Cirúrgicas",
    theme="soft"
)

demo.launch(share=True, debug=True)

🚀 Iniciando CardioBot com Aceleração por GPU T4...


llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8543bb03ac3d24dd8f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given
/usr/local/lib/python3.12/dist-packages/llama_cpp/llama.py:1242: RuntimeWarning: Detected duplicate leading "<|begin_of_text|>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


------- As próximas celulas são de verificação e ajustes --------

In [ ]:
# CÉLULA DE VERIFICAÇÃO DE INTEGRIDADE (CHECKSUM)
import hashlib

# Certifique-se de que a variável LLAMA_MODEL_PATH está definida com o caminho correto
# Esta variável vem da sua Célula 1.
model_file_path = LLAMA_MODEL_PATH

# Cria um objeto hash SHA256
sha256_hash = hashlib.sha256()

print(f"Calculando o checksum SHA256 para: {model_file_path}")
print("Isso pode levar um ou dois minutos para um arquivo grande...")

try:
    with open(model_file_path, "rb") as f:
        # Lê o arquivo em pedaços (chunks) para não sobrecarregar a memória RAM
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    calculated_checksum = sha256_hash.hexdigest()
    print("\n✅ Cálculo concluído!")
    print(f"\nChecksum Calculado: {calculated_checksum}")
    print(f"\nChecksum Oficial: 8ba9baf3a7345f705a11878397500fb25174034f0fd784e83aa4a96aaa47735f")

except FileNotFoundError:
    print(f"\nERRO: Arquivo não encontrado em '{model_file_path}'. Verifique o caminho.")
except Exception as e:
    print(f"\nOcorreu um erro: {e}")